In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting

In [2]:
order_products=pd.read_csv('/Users/jinglinhui/Desktop/Data_analytics_project/rawdata/order_products.csv')
orders=pd.read_csv('/Users/jinglinhui/Desktop/Data_analytics_project/rawdata/orders.csv')

## Group customer by their max order number

In [3]:
df=orders.groupby('user_id')['order_number'].aggregate('max').reset_index()

In [4]:
df['eval_set']=0

In [5]:
df.head(10)

,user_id,order_number,eval_set
0,1,11,0
1,2,15,0
2,3,13,0
3,4,6,0
4,5,5,0
5,6,4,0
6,7,21,0
7,8,4,0
8,9,4,0
9,10,6,0


## Joining grouped table to orders table by user_id,order_number

In [13]:
df1=pd.merge(orders,df, how='left', left_on=['user_id','order_number'], right_on=['user_id','order_number'])

In [14]:
df1.head(10)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2539329,1,1,2,8,NaN,NaN
1,2398795,1,2,3,7,15.0,NaN
2,473747,1,3,3,12,21.0,NaN
3,2254736,1,4,4,7,29.0,NaN
4,431534,1,5,4,15,28.0,NaN
5,3367565,1,6,2,7,19.0,NaN
6,550135,1,7,1,9,20.0,NaN
7,3108588,1,8,1,14,14.0,NaN
8,2295261,1,9,1,16,0.0,NaN
9,2550362,1,10,4,8,30.0,NaN


In [15]:
Total=df1[df1['eval_set']==0].count()

In [16]:
Total

order_id                  206209
user_id                   206209
order_number              206209
order_dow                 206209
order_hour_of_day         206209
days_since_prior_order    206209
eval_set                  206209
dtype: int64

## Fill in 'Prior' for the null value as they are not the lastest order

In [38]:
df1.eval_set.fillna('Prior',inplace=True)
df1

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2539329,1,1,2,8,NaN,Prior
1,2398795,1,2,3,7,15.0,Prior
2,473747,1,3,3,12,21.0,Prior
3,2254736,1,4,4,7,29.0,Prior
4,431534,1,5,4,15,28.0,Prior
...,...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0,Prior
3421079,1854736,206209,11,4,10,30.0,Prior
3421080,626363,206209,12,1,12,18.0,Prior
3421081,2977660,206209,13,1,12,7.0,Prior


## Join order_products on to group_max_order(df1) table

In [33]:
join=pd.merge(df1,order_products,how='left',on='order_id')
join.head(10)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set,product_id,add_to_cart_order
0,2539329,1,1,2,8,NaN,Prior,196.0,1.0
1,2539329,1,1,2,8,NaN,Prior,14084.0,2.0
2,2539329,1,1,2,8,NaN,Prior,12427.0,3.0
3,2539329,1,1,2,8,NaN,Prior,26088.0,4.0
4,2539329,1,1,2,8,NaN,Prior,26405.0,5.0
5,2398795,1,2,3,7,15.0,Prior,196.0,1.0
6,2398795,1,2,3,7,15.0,Prior,10258.0,2.0
7,2398795,1,2,3,7,15.0,Prior,12427.0,3.0
8,2398795,1,2,3,7,15.0,Prior,13176.0,4.0
9,2398795,1,2,3,7,15.0,Prior,26088.0,5.0


In [44]:
prior=join[join['eval_set']=="Prior"]
prior

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set,product_id,add_to_cart_order
0,2539329,1,1,2,8,NaN,Prior,196.0,1.0
1,2539329,1,1,2,8,NaN,Prior,14084.0,2.0
2,2539329,1,1,2,8,NaN,Prior,12427.0,3.0
3,2539329,1,1,2,8,NaN,Prior,26088.0,4.0
4,2539329,1,1,2,8,NaN,Prior,26405.0,5.0
...,...,...,...,...,...,...,...,...,...
33894093,2977660,206209,13,1,12,7.0,Prior,14197.0,5.0
33894094,2977660,206209,13,1,12,7.0,Prior,38730.0,6.0
33894095,2977660,206209,13,1,12,7.0,Prior,31477.0,7.0
33894096,2977660,206209,13,1,12,7.0,Prior,6567.0,8.0


## Check matching after the join and the order is latest

In [37]:
train = join[(join['product_id'].notnull()) & (join['eval_set']==0)]
train

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set,product_id,add_to_cart_order
59,1187899,1,11,4,8,14.0,0.0,196.0,1.0
60,1187899,1,11,4,8,14.0,0.0,25133.0,2.0
61,1187899,1,11,4,8,14.0,0.0,38928.0,3.0
62,1187899,1,11,4,8,14.0,0.0,26405.0,4.0
63,1187899,1,11,4,8,14.0,0.0,39657.0,5.0
...,...,...,...,...,...,...,...,...,...
33894101,272231,206209,14,6,14,30.0,0.0,40603.0,4.0
33894102,272231,206209,14,6,14,30.0,0.0,15655.0,5.0
33894103,272231,206209,14,6,14,30.0,0.0,42606.0,6.0
33894104,272231,206209,14,6,14,30.0,0.0,37966.0,7.0


## Check orders didn't match in the lastest order

In [35]:
test = join[join['product_id'].isnull()]
test

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set,product_id,add_to_cart_order
384,2774568,3,13,5,15,11.0,0.0,NaN,NaN
403,329954,4,6,3,12,30.0,0.0,NaN,NaN
464,1528013,6,4,3,16,22.0,0.0,NaN,NaN
1086,1376945,11,8,6,11,8.0,0.0,NaN,NaN
1161,1356845,12,6,1,20,30.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
33892542,2728930,206202,23,2,17,6.0,0.0,NaN,NaN
33892729,350108,206204,5,4,14,14.0,0.0,NaN,NaN
33893066,1043943,206206,68,0,20,0.0,0.0,NaN,NaN
33893290,2821651,206207,17,2,13,14.0,0.0,NaN,NaN


## Changing values to 'Train' and 'Test'

In [46]:
train.loc[train.eval_set == 0, 'eval_set'] = 'Train'
train

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set,product_id,add_to_cart_order
59,1187899,1,11,4,8,14.0,Train,196.0,1.0
60,1187899,1,11,4,8,14.0,Train,25133.0,2.0
61,1187899,1,11,4,8,14.0,Train,38928.0,3.0
62,1187899,1,11,4,8,14.0,Train,26405.0,4.0
63,1187899,1,11,4,8,14.0,Train,39657.0,5.0
...,...,...,...,...,...,...,...,...,...
33894101,272231,206209,14,6,14,30.0,Train,40603.0,4.0
33894102,272231,206209,14,6,14,30.0,Train,15655.0,5.0
33894103,272231,206209,14,6,14,30.0,Train,42606.0,6.0
33894104,272231,206209,14,6,14,30.0,Train,37966.0,7.0


In [48]:
test.loc[test.eval_set == 0, 'eval_set'] = 'Test'
test

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set,product_id,add_to_cart_order
384,2774568,3,13,5,15,11.0,Test,NaN,NaN
403,329954,4,6,3,12,30.0,Test,NaN,NaN
464,1528013,6,4,3,16,22.0,Test,NaN,NaN
1086,1376945,11,8,6,11,8.0,Test,NaN,NaN
1161,1356845,12,6,1,20,30.0,Test,NaN,NaN
...,...,...,...,...,...,...,...,...,...
33892542,2728930,206202,23,2,17,6.0,Test,NaN,NaN
33892729,350108,206204,5,4,14,14.0,Test,NaN,NaN
33893066,1043943,206206,68,0,20,0.0,Test,NaN,NaN
33893290,2821651,206207,17,2,13,14.0,Test,NaN,NaN


In [49]:
new_order=pd.concat([prior,test,train], axis=0)

In [56]:
new_order.head(10)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2539329,1,1,2,8,NaN,Prior
1,2539329,1,1,2,8,NaN,Prior
2,2539329,1,1,2,8,NaN,Prior
3,2539329,1,1,2,8,NaN,Prior
4,2539329,1,1,2,8,NaN,Prior
5,2398795,1,2,3,7,15.0,Prior
6,2398795,1,2,3,7,15.0,Prior
7,2398795,1,2,3,7,15.0,Prior
8,2398795,1,2,3,7,15.0,Prior
9,2398795,1,2,3,7,15.0,Prior


In [52]:
new_order=new_order.drop(['product_id','add_to_cart_order'],axis=1)
new_order.head(10)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2539329,1,1,2,8,NaN,Prior
1,2539329,1,1,2,8,NaN,Prior
2,2539329,1,1,2,8,NaN,Prior
3,2539329,1,1,2,8,NaN,Prior
4,2539329,1,1,2,8,NaN,Prior
5,2398795,1,2,3,7,15.0,Prior
6,2398795,1,2,3,7,15.0,Prior
7,2398795,1,2,3,7,15.0,Prior
8,2398795,1,2,3,7,15.0,Prior
9,2398795,1,2,3,7,15.0,Prior


In [60]:
new_order['eval_set'].ount()

In [61]:
check

order_id                  32434489
user_id                   32434489
order_number              32434489
order_dow                 32434489
order_hour_of_day         32434489
days_since_prior_order    30356421
eval_set                  32434489
dtype: int64